In [ ]:
!pip install tabulate

In [ ]:
import snowflake.connector as snow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime
import random
import statistics
import tabulate
from concurrent.futures import ThreadPoolExecutor, as_completed

conn_kwargs={}

def execute_and_print(query):
    cursor.execute(query)
    print(tabulate.tabulate(cursor.fetchall()))

def run_and_measure(count, mode):

    if mode =="std":
        query = """
                SELECT SearchEngineID, ClientIP, COUNT(*) AS c, SUM(IsRefresh), AVG(ResolutionWidth) FROM 
                BENCHMARK_FDN.HITS2_CSV
                WHERE SearchPhrase <> '' GROUP BY SearchEngineID, ClientIP ORDER BY c DESC LIMIT 10;
                """
        warehouse_query ="USE WAREHOUSE XSMALL"
    else:
        query = """
                SELECT SearchEngineID, ClientIP, COUNT(*) AS c, SUM(IsRefresh), AVG(ResolutionWidth) FROM 
                BENCHMARK_INTERACTIVE.HITS2_CSV
                WHERE SearchPhrase <> '' GROUP BY SearchEngineID, ClientIP ORDER BY c DESC LIMIT 10;
                """
        warehouse_query ="USE WAREHOUSE interactive_demo_b"
    timings = []
    with snow.connect(**conn_kwargs) as conn:
        with conn.cursor() as cur:
            cur.execute(warehouse_query)
            cursor.execute('ALTER SESSION SET USE_CACHED_RESULT = FALSE;')
            for i in range(count+1):
                t0 = time.time()
                cur.execute(query).fetchall()
                time_taken = time.time() - t0
                timings.append(time_taken)
                
    return timings[1:]
    


def plot_data(data, title, time_taken, color='blue'):
    # Separate titles and counts
    titles = [item[0] for item in data]
    counts = [item[1] for item in data]

    # Plot bar chart
    
    plt.figure(figsize=(12, 4))
    plt.bar(titles, counts, color=color)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Counts")
    plt.xlabel("Title")
    plt.title(title)
    plt.text(5, 240, f'Time taken: {time_taken:.4f} seconds', ha='right',va='top', fontdict={'size': 16})
    #plt.tight_layout()
    plt.show()

# Separate titles and counts
#titles = ['Run 1', 'Run 2', 'Run 3', 'Run 4', 'Run 5', 'Run 6', 'Run 7', 'Run 8']
counts_std = [0.1,0.15, 0.09, 0.12, 0.11, 0.13, 0.10, 0.14]
counts_iw = [0.05, 0.08, 0.07, 0.06, 0.09, 0.08, 0.07, 0.06]
# Plot bar chart

## Setting up connection to a Snowflake deployment, verify version

In [ ]:
config = { }
cursor = snow.connect(**config).cursor()
execute_and_print('select current_version();')
execute_and_print("show parameters like 'ENABLE_INTERACTIVE_WAREHOUSES' for account;")
execute_and_print("show parameters like 'ENABLE_INTERACTIVE_TABLE_DDL' for account;")
execute_and_print("show parameters like 'SHOW_INCLUDE_INTERACTIVE_TABLES' for account;")
query = """ USE DATABASE WILL_TEST; """
execute_and_print(query)

query = """ USE ROLE SYSADMIN;  """
execute_and_print(query)

## Create an interactive warehouse & Turn it on
![alt text](https://github.com/user-attachments/assets/edaac236-9a79-4519-8ca2-3091c7ad703c)


In [ ]:
query = """
CREATE or REPLACE INTERACTIVE WAREHOUSE interactive_demo_b
                WAREHOUSE_SIZE = 'XSMALL'
                MIN_CLUSTER_COUNT = 1
                MAX_CLUSTER_COUNT = 1
                COMMENT = 'Interactive warehouse demo';
"""
execute_and_print(query)
query = """
ALTER WAREHOUSE INTERACTIVE_DEMO_B RESUME;
"""
execute_and_print(query)

## Create an interactive table
![alt text](https://github.com/user-attachments/assets/2e0a7ba7-83d9-4b89-8184-cba5d590ebb2)

In [ ]:
print("Switch to demo database")
print(cursor.execute("USE DATABASE WILL_TEST").fetchall())

print("Use a standard warehouse for creating the interactive table's data")
print(cursor.execute("USE WAREHOUSE XLARGE").fetchall())

query = """
CREATE OR REPLACE INTERACTIVE TABLE 
WILL_TEST.BENCHMARK_INTERACTIVE.CUSTOMERS CLUSTER BY (CUSTOMER_ID)
AS
 SELECT * FROM WILL_TEST.BENCHMARK_FDN.CUSTOMERS
 
"""
execute_and_print(query)

## Attach interactive table to a warehouse
This is the process that pre-warms data
![alt text](https://github.com/user-attachments/assets/a08f7936-4da7-4415-88c4-f0ae7a2a60d7)

In [ ]:
query = """
USE DATABASE WILL_TEST;
"""
execute_and_print(query)

query = """
ALTER WAREHOUSE interactive_demo_b ADD TABLES(BENCHMARK_INTERACTIVE.HITS2_CSV);
"""
execute_and_print(query)

## Run queries with interactive warehouse
![alt text](https://github.com/user-attachments/assets/28aa2f08-562a-4e99-8785-e86d31c30e85)

In [ ]:
print("Use a standard warehouse for creating the interactive table's data")
cursor.execute("USE WAREHOUSE interactive_demo_b")
cursor.execute('use database WILL_TEST;')
cursor.execute('ALTER SESSION SET USE_CACHED_RESULT = FALSE;')

In [ ]:
query = """
SELECT Title, COUNT(*) AS PageViews
FROM BENCHMARK_INTERACTIVE.HITS2_CSV
WHERE CounterID = 62
  AND EventDate >= '2013-07-01'
  AND EventDate <= '2013-07-31'
  AND DontCountHits = 0
  AND IsRefresh = 0
  AND Title <> ''
  AND REGEXP_LIKE(Title, '^[\\x00-\\x7F]+$')
  AND LENGTH(Title) < 20
GROUP BY Title
ORDER BY PageViews DESC
LIMIT 10;
"""

start_time = time.time()
result = cursor.execute(query).fetchall()
end_time = time.time()
time_taken = end_time - start_time

plot_data(result, "Page visit analysis (Interactive)", time_taken)


## Compare to a standard warehouse
![alt text](https://github.com/user-attachments/assets/671a5d2a-cb0b-4da6-8cc2-8fe2a2c57c85)

In [ ]:
print("Use a standard warehouse for creating the interactive table's data")
cursor.execute("USE WAREHOUSE XSMALL")
cursor.execute('ALTER SESSION SET USE_CACHED_RESULT = FALSE;')
cursor.execute('use database WILL_TEST;')

In [ ]:
query = """
SELECT Title, COUNT(*) AS PageViews
FROM BENCHMARK_FDN.HITS2_CSV
WHERE CounterID = 62
  AND EventDate >= '2013-07-01'
  AND EventDate <= '2013-07-31'
  AND DontCountHits = 0
  AND IsRefresh = 0
  AND Title <> ''
  AND REGEXP_LIKE(Title, '^[\\x00-\\x7F]+$')
  AND LENGTH(Title) < 20
GROUP BY Title
ORDER BY PageViews DESC
LIMIT 10;
"""

start_time = time.time()
result = cursor.execute(query).fetchall()
end_time = time.time()
time_taken = end_time - start_time

plot_data(result, "Page visit analysis (Interactive)", time_taken, 'green')


## Run some queries concurrently

In [ ]:
runs = 4

counts_iw = run_and_measure(runs,"iw")
print(counts_iw)


counts_std = run_and_measure(runs,"std")
print(counts_std)

titles = [f"R{i}" for i in range(1, len(counts_iw)+1)]

x = np.arange(len(titles))  # the label locations
width = 0.35  # bar width

fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(x - width/2, counts_std, width, label="Standard", color="green")
ax.bar(x + width/2, counts_iw, width, label="Interactive", color="blue")

ax.set_ylabel("Latency")
ax.set_xlabel("Query run")
ax.set_title("Standard vs Interactive warehouse")
ax.set_xticks(x)
ax.set_xticklabels(titles)
ax.legend()
plt.show()

